In [1]:
from kafka import KafkaProducer
import pandas as pd
import json
from datetime import datetime, timedelta
import numpy as np

# Create a Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

class UserInteractionsGenerator:
    def __init__(self, user_profiles, product_data, num_hours=24 * 30):
        self.user_profiles = user_profiles
        self.product_data = product_data
        self.num_hours = num_hours
        self.start_time = datetime(2023, 11, 1, 0, 0, 0)
        self.interaction_id_counter = 1

    def generate_data(self):
        timestamps = [self.start_time + timedelta(hours=hour) for hour in range(self.num_hours)]
        user_ids = self.user_profiles['User_ID'].tolist()

        for timestamp in timestamps:
            for user_id in user_ids:
                interaction_type = np.random.choice(['product_search', 'product_view', 'add_to_cart', 'purchase'])
                product_id, additional_attributes = self.generate_interaction_details(interaction_type)

                data = {
                    'Interaction_ID': self.interaction_id_counter,
                    'Timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                    'User_ID': user_id,
                    'Interaction_Type': interaction_type,
                    'Product_ID': product_id,
                    'Product_Name': additional_attributes[0],
                    'Quantity': additional_attributes[1],
                    'Payment_Method': additional_attributes[2]
                }

                # Produce the data to the Kafka topic
                producer.send('User_Interactions', value=data)

                self.interaction_id_counter += 1

    def generate_interaction_details(self, interaction_type):
        if interaction_type == 'product_search':
            return '-', [np.random.choice(['laptop', 'phone', 'clothing', 'shoes', 'electronics']), 0, 'Not Applicable']
        elif interaction_type == 'product_view':
            product_id = np.random.choice(self.product_data['Product_ID'].tolist())
            product_name = self.product_data.loc[self.product_data['Product_ID'] == product_id, 'Product_Name'].values[0]
            return product_id, [product_name, 0, 'Not Applicable']
        elif interaction_type == 'add_to_cart':
            product_id = np.random.choice(self.product_data['Product_ID'].tolist())
            product_name = self.product_data.loc[self.product_data['Product_ID'] == product_id, 'Product_Name'].values[0]
            quantity = np.random.randint(1, 5)
            return product_id, [product_name, quantity, 'Not Applicable']
        elif interaction_type == 'purchase':
            product_id = np.random.choice(self.product_data['Product_ID'].tolist())
            product_name = self.product_data.loc[self.product_data['Product_ID'] == product_id, 'Product_Name'].values[0]
            quantity = np.random.randint(1, 5)
            payment_method = np.random.choice(['Credit Card', 'Debit Card', 'PayPal'])
            return product_id, [product_name, quantity, payment_method]

# Example usage

user_profiles = pd.read_csv(r'C:\Users\Olivia\Documents\Fall-2023\Big-Data-Technology\Big-Daa_Project\UserProfiles_data.csv')
product_data = pd.read_csv(r'C:\Users\Olivia\Documents\Fall-2023\Big-Data-Technology\Big-Daa_Project\ProductData_data.csv')

generator = UserInteractionsGenerator(user_profiles=user_profiles, product_data=product_data, num_hours=24 * 30)
generator.generate_data()


ModuleNotFoundError: No module named 'kafka'